In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import random as rand
from tqdm import tqdm
import random as rand

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

import sys
import os

# Agregar la carpeta 'src' al sys.path
src_path = os.path.abspath("../src")
if src_path not in sys.path:
    sys.path.append(src_path)
    import support_DecisionTreeRegressor as sp

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
#Lectura de csv
df = pd.read_csv("../data/df_te.csv")
df.sample(5)

In [ ]:
# Creacion de x (variables explicativas) e y (variable respuesta)
x=df.drop("price",axis=1)
y=df[["price"]]
x_train, x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [ ]:
# # Parámetros a evaluar
# params_arbol = {
#     'max_depth': [5, 10, 20],
#     'min_samples_split': [10, 20],
#     'min_samples_leaf': [50, 100, 150],
#     'max_leaf_nodes': [30, 40, 50, 60, 70, 100]
# }
# grid_search_arbol=GridSearchCV(DecisionTreeRegressor(),
#                                param_grid=params_arbol,
#                                cv=5,
#                                scoring="neg_mean_squared_error",
#                                n_jobs=-1)

# grid_search_arbol.fit(x_train,y_train)
# modelo_arbol_final=grid_search_arbol.best_estimator_

In [ ]:
y_train_pred=modelo_arbol_final.predict(x_train)
y_test_pred=modelo_arbol_final.predict(x_test)

In [ ]:
sp.metricas(y_train, y_train_pred, y_test, y_test_pred)

In [ ]:
sp.comparativa_graficos(y_test, y_test_pred)

### PROBAMOS A MEJORAR EL MODELO ELIMINANDO VARIABLES CON POCA REPERCUSIÓN

Calculamos la correlación de las variables con "price" para ver cuales pueden ser eliminadas por no ser representativas en el modelo

In [ ]:
sp.analizar_correlaciones(df, "price", threshold=0.05)

In [ ]:
# Excluir columnas irrelevantes según el análisis previo
columns_to_exclude = ['seller', 'offerType', 'abtest']
df_filtered = df.drop(columns=columns_to_exclude)

In [ ]:
# Definir las características (X) y la variable objetivo (y)
x2 = df_filtered.drop(columns=['price'])
y2 = df_filtered['price']

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.2, random_state=42)

In [ ]:
# Grid de hiperparámetros enfocado en regularización
param_grid = {
    'max_depth': [5, 7, 10],          # Reducir profundidad máxima
    'min_samples_split': [20, 50, 100],  # Aumentar muestras mínimas para dividir
    'min_samples_leaf': [10, 20, 50],    # Aumentar tamaño mínimo de nodos hoja
}

# Configurar GridSearchCV
model = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,  # Validación cruzada
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Entrenar el modelo
model.fit(x_train2, y_train2)

# Predecir en los datos de prueba
y_pred_test2 = model.predict(x_test2)
y_pred_train2 = model.predict(x_train2)

In [ ]:
sp.metricas(y_train2, y_pred_train2, y_test2, y_pred_test2)

In [ ]:
sp.comparativa_graficos( y_test2, y_pred_test2)

In [ ]:
# Definir el grid de parámetros
params_arbol = {
    'max_depth': [5, 10, 15, 20, 30, None],  # Explorar profundidad fija y sin límite
    'min_samples_split': [2, 10, 20, 50],  # Divisiones con más o menos datos
    'min_samples_leaf': [1, 5, 10, 20, 50],  # Nodos hoja con diferentes tamaños mínimos
    'max_leaf_nodes': [None, 20, 30, 50, 100, 200],  # Limitar o no el número de nodos hoja
}

# Configurar GridSearchCV
model2 = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid=params_arbol,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

# Entrenar el modelo con GridSearchCV
model2.fit(x_train2, y_train2)

# Mostrar los mejores parámetros y el error asociado
best_params = model2.best_params_

print(f"Mejores parámetros: {best_params}")

In [ ]:
y_pred_test3 = model2.predict(x_test2)
y_pred_train3 = model2.predict(x_train2)
sp.metricas(y_train2, y_pred_train3, y_test2, y_pred_test3)

In [ ]:
sp.comparativa_graficos( y_test2, y_pred_test3)

### PRUEBA DE MODELO LIMPIANDO DATOS CON PRECIOS OUTIERS

#### Ahora vamos a tratar de mejorar el modelo por última vez eliminando precios muy extremos que podemos calificar como outiers que generan ruido en el modelo

In [ ]:
# Cálculo del rango intercuartílico (IQR)
q1 = df['price'].quantile(0.25)  # Primer cuartil
q3 = df['price'].quantile(0.75)  # Tercer cuartil
iqr = q3 - q1

# Definir límites para identificar outliers (enfoque más estricto)
upper_bound = q3 + 3 * iqr

# Filtrar los outliers superiores
outliers = df[df['price'] > upper_bound]

# Visualizar los resultados
plt.figure(figsize=(12, 6))

# Boxplot mostrando los límites y outliers
plt.subplot(1, 2, 1)
sns.boxplot(data=df, x='price', color='lightblue')
plt.axhline(upper_bound, color='red', linestyle='--', label=f'Límite superior ({upper_bound:.2f})')
plt.title('Boxplot con Límite para Outliers')
plt.legend()

# Histograma resaltando los outliers
plt.subplot(1, 2, 2)
sns.histplot(df['price'], bins=30, kde=True, color='lightblue')
plt.axvline(upper_bound, color='red', linestyle='--', label=f'Límite superior ({upper_bound:.2f})')
plt.title('Histograma con Límite para Outliers')
plt.legend()

plt.tight_layout()
plt.show()

# Mostrar información sobre los outliers
print("Valores considerados outliers:")
print(outliers)

print(f"\nUmbral superior más estricto para considerar outliers: {upper_bound:.2f}")


In [ ]:
# Ver valores atipicos de "price"
limite_inferior = 300
limite_superior = 35000

df_filtered_copy=df_filtered.copy()

# Filtro los valores fuera de rango
valores_fuera_rango = df_filtered_copy[(df_filtered_copy['price'] < limite_inferior) | (df_filtered_copy['price'] > limite_superior)]

# Reemplazo los valores fuera de rango por NaN
df_filtered_copy.loc[df_filtered_copy['price'] < limite_inferior, 'price'] = np.nan
df_filtered_copy.loc[df_filtered_copy['price'] > limite_superior, 'price'] = np.nan

# Elimino las filas con NaN en la columna 'price' en el DataFrame original
df_filtered2 = df_filtered_copy.dropna(subset=['price'])

# Ver la forma del DataFrame con valores fuera de rango
valores_fuera_rango.shape

In [ ]:
# Definir las características (X) y la variable objetivo (y)
x3 = df_filtered2.drop(columns=['price'])
y3 = df_filtered2['price']

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train3, x_test3, y_train3, y_test3 = train_test_split(x3, y3, test_size=0.2, random_state=42)

In [ ]:
# Grid de hiperparámetros enfocado en regularización
param_grid = {
    'max_depth': [5, 10, 20, 30, None],  # Explorar profundidad fija y sin límite
    'min_samples_split': [2, 10, 20, 50],  # Divisiones con más o menos datos
    'min_samples_leaf': [1, 5, 10, 20, 50],  # Nodos hoja con diferentes tamaños mínimos
    'max_leaf_nodes': [None, 20, 30, 50, 100],  # Limitar o no el número de nodos hoja
}

# Configurar GridSearchCV
model = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=param_grid,
    cv=10,  # Validación cruzada
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Entrenar el modelo
model.fit(x_train3, y_train3)

# Predecir en los datos de prueba
y_pred_test4 = model.predict(x_test3)
y_pred_train4 = model.predict(x_train3)

In [ ]:
sp.metricas(y_train2, y_pred_train3, y_test2, y_pred_test3)

In [ ]:
sp.metricas(y_train3, y_pred_train4, y_test3, y_pred_test4)

In [ ]:
sp.comparativa_graficos( y_test3, y_pred_test4)

#### COMPARATIVA DE METRICAS